# Testing everything

In [1]:
import pandas as pd
from datetime import datetime
from googlesearch import search
import time

In [2]:
fund_list_file = 'fundlist.txt'
df = pd.read_csv(fund_list_file, sep='\t',header=None,parse_dates=True,usecols=[1,3,4,8])
df = df.rename(columns={1:'name',3:'currency',4:'category',8:'risk'})

In [3]:
def mysearch(search_str:str):
    search_result = search(search_str, tld="com", num=20, stop=20, pause=5)
    return list(search_result)

dfi = pd.read_csv('fund_symbols.csv')
data = []
try:
    for i in range(122,400):
        x = None
        search_result = mysearch(df.loc[i]['name']+" site:morningstar.pt")
        for url in search_result:
            if url[:58] == "https://www.morningstar.pt/pt/funds/snapshot/snapshot.aspx":
                url = url[:72]
                x = pd.read_html(url)[3]
        if x is None:
            search_result = mysearch(df.loc[i]['name']+" site:morningstar.es")
            for url in search_result:
                if url[:58] == "https://www.morningstar.es/es/funds/snapshot/snapshot.aspx":
                    url = url[:72]
                    x = pd.read_html(url)[3]    
        if x is None:
            print("'{}' Not Found!".format(df.loc[i]['name']))
        else:
            symbol = x.loc[4][2]+":"+x.loc[1][2][:3]
            print("{} {} '{}' {}".format(i, symbol,df.loc[i]['name'],url))
            data.append((i, symbol,df.loc[i]['name'],df.loc[i]['currency'],df.loc[i]['risk'],df.loc[i]['category'],url))
except Exception as e:
    print(e)
    print('line {}'.format(i))
    
dfd = pd.DataFrame(data=data,columns=['ID','symbol','name','currency','risk','category','url'])
pd.concat([dfi,dfd]).to_csv('fund_symbols.csv',index=False)

104 LU1496042869:EUR 'Carmignac Pf Global Bond A EUR Acc' https://www.morningstar.pt/pt/funds/snapshot/snapshot.aspx?id=F00000YNLV
105 IE00B7BTH691:USD 'Neuberger Berman Strat Inc A USD MD' https://www.morningstar.pt/pt/funds/snapshot/snapshot.aspx?id=F00000Q070
106 LU0562246883:USD 'JPM Emerg Mkts Inv Grd Bd D (acc) USD' https://www.morningstar.pt/pt/funds/snapshot/snapshot.aspx?id=F00000ONDG
107 ES0159201013:EUR 'March International Iberia A EUR' http://tools.morningstar.es/es/stockreport/default.aspx?tab=3&vw=story&SecurityToken=0P00007OUD%5D3%5D0%5DE0WWE%24%24ALL&Id=0P00007OUD&ClientFund=0&CurrencyId=EUR&story=LSE20180802070010_13740626
108 LU1670725347:USD 'M&G (Lux) Optimal Income A H USD Acc' https://www.morningstar.pt/pt/funds/snapshot/snapshot.aspx?id=F000010L36
109 LU0552385295:USD 'MS INVF Global Brands Eq Inc A' https://www.morningstar.es/es/news/31278/%C2%BFqui%C3%A9n-lo-hizo-mejor-en-renta-variable-en-2011.aspx
110 IE00BH65QK91:EUR 'Vanguard Glbl Sh-Tm Bd Idx Inv EURH Acc